In [72]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from skimage import io, util
from PIL import ImageFont, ImageDraw, Image ,ImageFilter
import os
import numpy as np
import math
import cv2
import string
import random
import calendar
import locale
# import imutils

In [73]:
top_padding = 150
bottom_padding = 150
left_padding = 200
right_padding = 200
def preprocess_image(image, angle):
    x = np.uint8(image)
    image_array = np.asarray(x)
    height, width = image_array.shape[:2]
    center = (width / 2, height / 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    abs_cos = abs(rotation_matrix[0, 0])
    abs_sin = abs(rotation_matrix[0, 1])
    new_width = int(height * abs_sin + width * abs_cos)
    new_height = int(height * abs_cos + width * abs_sin)
    rotation_matrix[0, 2] += (new_width - width) / 2
    rotation_matrix[1, 2] += (new_height - height) / 2
    rotated_image = cv2.warpAffine(image_array, rotation_matrix, (new_width, new_height))

    # Convert black pixels to gray
    black_pixel = np.asarray([0, 0, 0], dtype=rotated_image.dtype)
    mask = np.all(rotated_image == black_pixel, axis=2)
    # generate random RGB values between 0 and 255
    r = random.randint(0, 128)
    g = random.randint(0, 128)
    b = random.randint(0, 128)

    rotated_image[mask] = [r,g,b]
    
   
    rotated_image = cv2.copyMakeBorder(rotated_image, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=[r,g,b])
    rotated_image = Image.fromarray(rotated_image)
    rotated_image = rotated_image.resize((width, height))
    # rotated = imutils.rotate_bound(image, angle)
    # rotated_image = imutils.rotate_bound(image_array, angle)

    return rotated_image

In [74]:
def ImageGenerate(image_path,  iteration, extension, save_dir):
    # define the data augmentation settings
    # First test 
        # datagen = ImageDataGenerator(
        #     rotation_range=20,
        #     width_shift_range=0.1,
        #     height_shift_range=0.1,
        #     shear_range=0.2,
        #     zoom_range=0.2,
        #     horizontal_flip=False,
        #     fill_mode='constant',
        #     cval=128)
    # Second Test
    # datagen = ImageDataGenerator(
    #         rotation_range=20,
    #         zoom_range=[1,1.5],
    #         shear_range=0.2,
    #         horizontal_flip=False,
    #         fill_mode='constant',
    #         cval=128,
    #         width_shift_range=0.0,
    #         height_shift_range=0.0)
    # Third Test
    datagen = ImageDataGenerator(
            horizontal_flip=False,
            preprocessing_function=lambda x: preprocess_image(x,angle=random.randint(-20, 20))
    )
    # load the original image
    img = load_img(image_path)
    # convert the image to a numpy array
    x = img_to_array(img)

    # reshape the array to a single sample with a single channel
    x = x.reshape((1,) + x.shape)
    # generate new samples using data augmentation
    i = 0
    try:
        for batch in datagen.flow(x, batch_size=1, save_to_dir=save_dir ,save_prefix='my_image_{}_{}'.format(iteration,i), save_format=extension):
            i += 1
            if i > 9:
                break 
    except Exception as e:
        print(f"An error occurred: {e}")   

In [75]:
def put_noise_to_image(save_dir, iteration):
    # Define the path to the directory containing the images

    # Define the types of noise and their parameters
    noise_types = ['gaussian', 'salt', 'pepper', 'speckle']
    noise_params = [{'mean': 0, 'var': 0.02}, {'amount': 0.01}, {'amount': 0.01}, {'mean': 0, 'var': 0.01}]
    filename_pattern = 'my_image_{}_'.format(iteration)
    matching_files = [filename for filename in os.listdir(save_dir) if filename.startswith(filename_pattern)]
    # Loop over all the images in the directory
    for img_path in matching_files:
        # Load the image
        img = io.imread(os.path.join(save_dir, img_path))

        # Add different types of random noise to the image
        noisy_imgs = []
        for i, noise_type in enumerate(noise_types):
            if noise_type == 'salt' or noise_type == 'pepper':
                noisy_img = util.random_noise(img, mode=noise_type, **noise_params[i])
            else:
                noisy_img = util.random_noise(img, mode=noise_type, clip=True, **noise_params[i])
            noisy_imgs.append(noisy_img)

        # Save the noisy images
        for i, noisy_img in enumerate(noisy_imgs):
            io.imsave(os.path.join(save_dir, 'noisy_{}_{}'.format(noise_types[i], img_path)), noisy_img)

In [76]:
def blur_image(save_dir, iteration):
    filename_pattern = 'my_image_{}_'.format(iteration)
    matching_files = [filename for filename in os.listdir(save_dir) if filename.startswith(filename_pattern)]
    for img_path in matching_files:
        img = Image.open(os.path.join(save_dir, img_path))
        # Apply some blur to the image
        img = img.filter(ImageFilter.GaussianBlur(radius=1))

        # # Convert the image to a numpy array
        # img_array = np.array(img)

        # # Add some noise to the image
        # noise = np.random.normal(loc=0, scale=15, size=img_array.shape)
        # noisy_img_array = np.clip(img_array + noise, 0, 255).astype(np.uint8)

        # # Convert the numpy array back to an image
        # noisy_img = Image.fromarray(noisy_img_array)
        
        # Save the noisy image
        img.save(os.path.join(save_dir, 'blur_{}'.format(img_path)))

In [77]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

In [78]:
def painte_img_pil(image, x0, y0, x2, y2):
    numpy_array = np.asarray(image)
    mask = np.zeros(numpy_array.shape[:2], dtype="uint8")
    x1, y1 = x2, y0
    x3, y3 = x0, y2
    # (x0, y0) is the top-left vertex of the bounding box
    # (x1, y1) is the top-right vertex of the bounding box
    # (x2, y2) is the bottom-right vertex of the bounding box
    # (x3, y3) is the bottom-left vertex of the bounding box
    x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
    x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
    #For the line thickness, we will calculate the length of the line between 
    #the top-left corner and the bottom-left corner.
    thickness = int(math.sqrt((x2 - x1)**2 + (y2 - y1)**2))

    #Define the line and inpaint
    cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,    
    thickness)

    inpainted_img = cv2.inpaint(numpy_array, mask, 7, cv2.INPAINT_NS)
    pil_image = Image.fromarray(inpainted_img)
    
    return pil_image

In [79]:
def get_facture_num():
    # Generate random letters
    letters = ''.join(random.choices(string.ascii_uppercase, k=1))
    # Generate random digits
    digits = ''.join(random.choices(string.digits, k=10))
    # Combine the letters and digits into a single string
    result = letters + digits
    return result


In [80]:
def generate_random_email(firstname, last_name):
    provider = random.choice(['gmail.com', 'outlook.com', 'hotmail.com','gmail.fr', 'outlook.fr', 'hotmail.fr'])
    username = f"{firstname.lower()}.{last_name.lower()}"
    email_address = f"{username}@{provider}"
    return email_address

In [81]:
def get_date_info(date_string):
    # Set the locale to French
    locale.setlocale(locale.LC_TIME, 'fr_FR')
    # Convert the date string back into a date object
    date_object = datetime.strptime(date_string, '%d-%m-%Y')
    # Get the month as a two-digit string
    month = date_object.strftime('%m')
    year = date_object.strftime('%Y')
    month_name = calendar.month_name[int(month)]
    return month_name,year

In [82]:
def get_dossier_num():
    # Generate random letters
    letters = ''.join(random.choices(string.ascii_uppercase, k=2))
    # Generate random digits
    digits = ''.join(random.choices(string.digits, k=7))
    # Combine the letters and digits into a single string
    result = letters + digits
    return result


In [83]:
# iteration = 6
# save_dir = 'TestImage'
# img_path = 'TestImage\TestBlur\my_image_4_0_0_8359.jpg'
# filename_pattern = 'my_image_{}_'.format(iteration)
# for i in range(iteration):
#     img = Image.open( img_path)
#     # Apply some blur to the image
#     # img = img.filter(ImageFilter.GaussianBlur(radius=1))

#     # Convert the image to a numpy array
#     img_array = np.array(img)

#     # # Add some noise to the image
#     noise = np.random.normal(loc=0, scale=15, size=img_array.shape)
#     noisy_img_array = np.clip(img_array + noise, 0, 255).astype(np.uint8)

#     # # Convert the numpy array back to an image
#     noisy_img = Image.fromarray(noisy_img_array)
    
#     # Save the noisy image
#     img.save('TestImage/TestBlur/dddur1234.jpg')